# Управление зависимостями. Библиотеки



## История dotnet

![History.png](History.png)

https://time.graphics/line/291016

Common Language Infrastructure - спецификация для создания инфраструктуры, при которой исполнение кода платформо-независимо

### Ключевые моменты
1) Появился .net framework (первая реализация CLI,  CLR - виртуальная машина)
2) Создали .net core (следующая реализация CLI, CoreCLR - виртуальная машина)
3) Сформировали .net standard
4) Потом решили всех запутать и назвали просто .net (типо замена .net framework)

### .NET Standard

![](standard.jpg)

## Управление зависимостями

Изучаем зависимости библиотеки, которую хотим установить:

Пример:

### Наш проект

```XML
<PropertyGroup>
    <OutputType>WinExe</OutputType>
    <TargetFramework>net framework 4.5.1</TargetFramework>
    <UseWPF>true</UseWPF>
</PropertyGroup>
```

### Хотим установить либу, которая бы парсила json
### Не получится

![](nuget.png)

### Все норм

![](ok.png)

## Как устанавливать зависимости

Nuget - Visual Studio (В Rider тоже самое)

![](nuget-vs.png)
![](browser-vs.png)

### VS code - можно установить расширение NuGet Gallery 

# Unit Testing And Mocking

Тестирование - испытание программного продукта, имеющий своей целью проверку соответствия между    
реальным поведением программы и её ожидаемым поведением на конечном наборе тестов, выбранных определённым образом.

Unit Тестирование - тестирование маленьких блоков функциональности некоторого объекта. ВАЖНОЙ ЧАСТЬЮ юнит тестов        
является то, что они тестируют куски функционала изолировано от всех внешних систем. Из-за этого они должны быстро    
запускаться и проходить.

## Nunit

### Установка (В случае Visual Studio может понадобиться установить NUnit3TestAdapter)
![](2024-03-29-18-39-43.png)


### DOCUMENTATION : https://docs.nunit.org/articles/nunit/intro.html

## Основные аттрибуты

### [TestFixture] 

Ставится у классов. Означает, что данный класс содержит в себе тесты.

### [Test]

Ставится у методов. Означает, что данный метод является тестом.

```c#
[TestFixture]
public class Testing
{
    [Test]
    public void ALWAYS_FALLING_TEST()
    {
        Assert.That(0, Is.EqualTo(1));
    }
    
    [Test]
    public void ALWAYS_PASSING_TEST()
    {
        Assert.That(0, Is.Not.EqualTo(1));
    }
}
```

### Запустим это (в IDE сбоку на кнопки можно тыкнуть, в vs code через комманду dotnet test)
![](2024-03-29-19-01-46.png)

### [SetUp], [TearDown]

Ставятся у методов. Методы помеченные этими аттрибутами, выполнятся до тест-кейса и после тест-кейса соответственно.

```c#
private int _tearDownAndSetUpValue = 1;

[SetUp]
public void SetUpValue()
{
    Console.WriteLine($"Value before SetUp: {_tearDownAndSetUpValue}");
    _tearDownAndSetUpValue = 2;
}

[Test]
public void CheckValueInBetween()
{
    Console.WriteLine($"Value after SetUp and before TearDown: {_tearDownAndSetUpValue}");
    _tearDownAndSetUpValue = 100500;
}

[TearDown]
public void TearDownValue()
{
    Console.WriteLine($"Value before TearDown: {_tearDownAndSetUpValue}");
    _tearDownAndSetUpValue = 1;
}
```

Value before SetUp: 1       
Value after SetUp and before TearDown: 2        
Value before TearDown: 100500       

### [OneTimeSetUp], [OneTimeTearDown]
Тоже самое, что и предыдущие аттрибуты. Только выполняются один раз, в начале и конце test-fixture

```c#
private int _setOnceValue = 1;

[OneTimeSetUp]
public void SetUpOnce()
{
    _setOnceValue *= -1;
}

[Test]
public void TestOnce()
{
    // Value set Once: -1
    Console.WriteLine($"Value set Once: {_setOnceValue}");
}

[Test]
public void TestOnceOther()
{
    // Value set Once other test case: -1
    Console.WriteLine($"Value set Once other test case: {_setOnceValue}");
}
```

### [TestCaseSource]
Иногда можно написать тест, который умеет работать с набором однородных данных.     
Для того, чтобы не писать множество тестов с одинаковым кодом, можно использовать этот аттрибут

```c#
[TestCaseSource(nameof(DivideCases))]
public void DivideTest(int n, int d, int q)
{
    Assert.That(q, Is.EqualTo(n / d));
}

public static object[] DivideCases =
{
    new object[] { 12, 3, 4 },
    new object[] { 12, 2, 6 },
    new object[] { 12, 4, 3 }
};
```

Из плюсов то, что это все дробится в отдельные тесты на самом деле:

![](2024-03-29-19-41-46.png)

## Asserts

Assert состоит из двух частей:

Actual (первое значение) : Что у нас получилось в конце теста.    
Expression Resolver (второе значение) : с чем и как мы сравниваем

```c#
Assert.That(1, Is.EqualTo(1)); // Проверка на равенство
```
Оператор Not (на самом деле свойство) инвертирует условие
```c#
Assert.That(1, Is.Not.EqualsTo(1)) // Проверка на неравенство
```

EqualsTo работает с любыми объектами (так как у них есть метод такой).

*** В чем может быть подвох ***

```c#
[Test]
public void Asserts_OTHER()
{
    int[] GetInts()
    {
        return new int[] { 1, 2, 3, 4 };
    }
    
    Assert.That(GetInts(), Has.Length.EqualTo(4)); // OK int[] есть свойство Length
    Assert.That(GetInts(), Has.Count.EqualTo(4));  // NOT OK int[] свойства Count нет

    // Есть только 1 элемент ( Exactly(1) ) равный единице ( EqualTo(1) )
    Assert.That(GetIntegers(), Has.Exactly(1).EqualTo(1));
}

[Test]
public void Asserts_EqualityCollection()
{
    Assert.That(new[] {1, 2, 3, 4}, Is.EquivalentTo(new[] {1, 2, 3, 4}));
    Assert.That(new[] {1, 2, 3, 4}, Has.Member(1).And.Member(2));
    // Assert.That(new[] {1, 2, 3, 4}, Has.Member(1).And.Not.Member(3)); Will fail
}
```

### Classic Asserts

Еще есть классические ассерты по типу Assert.IsTrue, Assert.Throws и т.д

## Работа с тестовыми файлами

Копируем в \$YOUR_TESTING_PROJECT_FILE.csproj\$ следующее

```c#
<ItemGroup>
    <None Include="$(SolutionDir)\$TEST_PROJECT_DIR$\$TEST_DIR$\**" CopyToOutputDirectory="Always" />
</ItemGroup>
```

А в коде самих тестов пишем следующее:

```C#
var directory = Path.GetDirectoryName(System.Reflection.Assembly.GetExecutingAssembly().Location);      
var testFolder = Path.Combine(directory!, "YOUR_TESTING_SOLUTION_FOLDER");     
var testPath = Path.Combine(testFolder, "YOUR_TESTING_FILE_NAME");        
```

testPath - путь до файла с тестовыми данными

## Mocking

```c#
public interface IDependency
{
    // Очень тяжелая зависимость (база данных, что-то из файловой системы и т.д)
    object GetImportantObject();
}

private class SomeDependency : IDependency
{
    public object GetImportantObject() => throw new NotImplementedException();
}



public void EmulateSomeWork(IDependency dependency)
{
    var obj = dependency.GetImportantObject();
    
    // Что-нибудь делаем с этим obj
}
```


### Простой пример:

```c#
public interface ITextProvider
{
    string GetText(int id);
}

public class TextWorker
{
    public string Execute(ITextProvider textProvider)
    {
        var result = textProvider.GetText(1);
        // Это очень важная работа с result,
        // а именно добавить в конец is ready
        return result + " is ready";
    }
}
```

# FIRE IT UP

```c#
...
using NSubstitute; // Mocking Framework

[TestFixture]
public class MockingEasy
{
    [Test]
    public void Testing()
    {
        const string textAsset = "Very important asset";

        var textProviderMock = Substitute.For<ITextProvider>();
        textProviderMock.GetText(Arg.Any<int>()).Returns(textAsset);

        var actualResult = new TextWorker().Execute(textProviderMock);

        Assert.That(actualResult, Is.EqualTo(textAsset + " is ready"));
    }
}
```

### ЧТО ТОЛЬКО ЧТО ПРОИЗОШЛО

Создали мок нашего класса. Пока что это просто заглушка.    
Если вызовем GetText, то нам вернется default(string).
```c#
var textProviderMock = Substitute.For<ITextProvider>();
```

Определяем поведение, которое происходит при вызове функции.    
Arg.Any() говорит о том, что при вызове с любым аргументом возвращать значение в Returns
```c#
textProviderMock.GetText(Arg.Any<int>()).Returns(textAsset);
```

Можно указывать также условия на аргументы. Если будет выполнено условие, то будет возвращено значение
```c#
textProviderMock.GetText(Arg.Is<int>(x => x <= 2)).Returns(textAsset);
```

При этом условия на аргументы можно сочетать друг с другом

```c#
public string Execute(ITextProvider textProvider, int position = 5)
{
    var result = textProvider.GetText(position);
    return result + " is ready";
}

{
  const string textAsset = "Very important asset";
  const string otherAsset = "Other important asset";

  var textProviderMock = Substitute.For<ITextProvider>();
  textProviderMock.GetText(Arg.Is<int>(x => x <= 2)).Returns(textAsset);
  textProviderMock.GetText(Arg.Is<int>(x => x > 2)).Returns(otherAsset);

  var firstResult = new TextWorker().Execute(textProviderMock, 0);
  var otherResult = new TextWorker().Execute(textProviderMock, 3);

  Console.WriteLine(firstResult); // Very important asset is ready
  Console.WriteLine(otherResult); // Other important asset is ready
}
```

## Сложный пример

```C#
public interface IFactoryObject
{
    string GetMessage();
}

public enum ObjectType
{
    None,
    A,
    B,
}

public interface IFactory
{
    IFactoryObject? GetObject(ObjectType typeToCreate);
}

private const string MockedMessageA = "MOCKED MESSAGE FROM AObject";
private const string MockedMessageB = "MOCKED MESSAGE FROM BObject";
private const string MockedMessageDefault = "MOCKED MESSAGE FROM Default";

var mockFactory = Substitute.For<IFactory>();
        
mockFactory.GetObject(Arg.Any<ObjectType>())
    .Returns(info =>
    {
        if (info.ArgAt<ObjectType>(0) == ObjectType.A)
        {
            var mockedAObj = Substitute.For<IFactoryObject>();
            mockedAObj.GetMessage().Returns(MockedMessageA);

            return mockedAObj;
        }

        if (info.ArgAt<ObjectType>(0) == ObjectType.B)
        {
            var mockedBObj = Substitute.For<IFactoryObject>();
            mockedBObj.GetMessage().Returns(MockedMessageB);

            return mockedBObj;
        }

        var mockedDefaultObject = Substitute.For<IFactoryObject>();
        mockedDefaultObject.GetMessage().Returns(MockedMessageDefault);

        return mockedDefaultObject;
    });

var mockedA = mockFactory.GetObject(ObjectType.A);
Assert.That(mockedA!.GetMessage(), Is.EqualTo(MockedMessageA));

var mockedB = mockFactory.GetObject(ObjectType.B);
Assert.That(mockedB!.GetMessage(), Is.EqualTo(MockedMessageB));

var mockedDefault = mockFactory.GetObject(ObjectType.None);
Assert.That(mockedDefault!.GetMessage(), Is.EqualTo(MockedMessageDefault));
```

## ВАЖНАЯ ИНФОРМАЦИЯ

### НЕ мокайте не абстрактные (виртуальные осторожно) методы

### НЕ мокайте internal классы и методы

### Лучше не мокайте ничего кроме интерфейсов